In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import condorgmm
import condorgmm.eval.metrics
import condorgmm.data
import datetime as dt
from tqdm import tqdm
from condorgmm import Pose
import fire
import condorgmm.object_tracking.integrated_object_tracking
import condorgmm.object_tracking.em_object_tracking
import condorgmm.object_tracking.low_frame_rate

In [ ]:

scene=None
dataset=None
object_index=None
experiment_name=None
max_T=None
mode="integrated"
debug=False
stream_rerun=False
downscale=1
upscale=1
frame_rate=1
log_period=1

mode="integrated"
dataset="ycbineoat"
scene="cracker_box_reorient"
min_T = 100
max_T = 300
stream_rerun=True
downscale=4



In [ ]:
if mode == "integrated":
    initialize = condorgmm.object_tracking.integrated_object_tracking.initialize
    update = condorgmm.object_tracking.integrated_object_tracking.update
    rr_log = condorgmm.object_tracking.integrated_object_tracking.rr_log
elif mode == "em":
    initialize = condorgmm.object_tracking.em_object_tracking.initialize
    update = condorgmm.object_tracking.em_object_tracking.update
    rr_log = condorgmm.object_tracking.em_object_tracking.rr_log
elif mode == "low_frame_rate":
    initialize = condorgmm.object_tracking.low_frame_rate.initialize
    update = condorgmm.object_tracking.low_frame_rate.update
    rr_log = condorgmm.object_tracking.low_frame_rate.rr_log
else:
    raise ValueError(f"Invalid mode: {mode}")

# Generate unique run ID using timestamp
if experiment_name is None:
    experiment_name = ""
timestamp = dt.datetime.now(dt.UTC).strftime("%Y-%m-%d-%H-%M-%S")
experiment_name = f"{experiment_name}_{timestamp}"
print(f"Experiment name: {experiment_name}")

results_dir = (
    condorgmm.get_root_path()
    / "results"
    / f"object_pose_tracking_{dataset}_{mode}_{experiment_name}"
)
output_file = results_dir / "output.txt"

results_dir.mkdir(parents=True, exist_ok=True)
# make output file
with open(output_file, "w") as f:
    f.write(f"Running object pose tracking for {dataset} dataset\n")
    f.write(f"Experiment name: {experiment_name}\n")
    f.write(f"Mode: {mode}\n")
    f.write(f"Debug: {debug}\n")
    f.write(f"Downscale: {downscale}\n")
    f.write(f"Max T: {max_T}\n")

if dataset == "ycbineoat":
    DatasetVideoType = condorgmm.data.YCBinEOATVideo
elif dataset == "ycbv":
    DatasetVideoType = condorgmm.data.YCBVVideo
elif dataset == "ycbv_test":
    DatasetVideoType = condorgmm.data.YCBTestVideo
else:
    raise ValueError(f"Invalid dataset: {dataset}")

if scene is None:
    scene_names = DatasetVideoType.SCENE_NAMES
elif isinstance(scene, str):
    scene_names = [scene]
elif isinstance(scene, list):
    scene_names = scene
elif isinstance(scene, int):
    scene_names = [scene]
else:
    raise ValueError(f"Invalid scene argument: {scene} , type: {type(scene)}")

print("This will run camera pose tracking for the following scenes:")
print(scene_names)


In [ ]:
scene_name = scene_names[0]

In [ ]:
video = DatasetVideoType(scene_name).downscale(downscale).upscale(upscale)
max_T_local = min(len(video), max_T) if max_T is not None else len(video)
timesteps = list(range(min_T, max_T_local, frame_rate))

print("\tLoading video...")

frames = video.load_frames(timesteps)

print(f"Running camera pose tracking for YCB-inEOAT dataset scene {scene_name}")
object_indices = (
    range(len(video[0].object_ids)) if object_index is None else [object_index]
)
current_object_index = object_indices[0]

In [ ]:
minT = 100
maxT = 300


condorgmm.rr_init(f"object pose tracking {scene_name} {current_object_index}")
print(
    f"Running camera pose tracking for YCB-inEOAT dataset scene {scene_name} and object {current_object_index}"
)

print("\tInitializing the object tracker...")

object_id = video[0].object_ids[current_object_index]
object_mesh = video.get_object_mesh_from_id(object_id)
object_name = video.get_object_name_from_id(object_id)

camera_pose_0, object_pose_0, og_state, debug_data = initialize(
    frame=video[minT],
    object_mesh=object_mesh,
    object_idx=current_object_index,
    debug=debug,
    seed=1
)

camera_poses_over_time = [camera_pose_0]
object_poses_over_time = [object_pose_0]

aggregated_debug_data = []

state = og_state

if stream_rerun:
    rr_log(state, frames[timesteps[0]], 0)
print("\tInitialized.")

In [ ]:


state.condor_state.state.matter

In [ ]:
print("\tRunning tracking on all subsequent frames...")

for timestep in tqdm(timesteps[1:]):
    camera_pose, object_pose, state, debug_data = update(
        state, video[timestep], timestep=timestep, debug=debug
    )
    camera_poses_over_time.append(camera_pose)
    object_poses_over_time.append(object_pose)
    aggregated_debug_data.append(debug_data)

    if stream_rerun and timestep % log_period == 0:
        rr_log(state, video[timestep], timestep, do_log_frame=False)


